In [1]:
%%pyspark
df = spark.read.load('abfss://adlsynapsecontainer@adlstoragesynapse203.dfs.core.windows.net/synapse/workspaces/synapse-ws-203/Stage/Customer Profile/Customer Profile.json', format='json')
display(df.limit(2))


In [2]:
from pyspark.sql.functions import lit,  array, concat, col

df1 = df.na.fill("")
# df1 = df1.withColumn("Full Name", array("FirstName", "LastName"))

# arr = df1.select(concat(col("FirstName"),lit(' '),col("MiddleName"),lit(' '),col("LastName")))
# print(arr)

# df1 = df1.withColumn("Full Name", concat(col("FirstName"),lit(' '),col("MiddleName"),lit(' '),col("LastName")))

df1 = df1.withColumn("Full Name", concat("FirstName", lit(' '), "MiddleName", lit(' '), "LastName"))

display(df1.limit(2))



In [3]:
from datetime import date
from pyspark.sql.functions import year, month, dayofmonth, when
import numpy as np

curr_year = date.today().year
curr_month = date.today().month
curr_day = date.today().day
# print(curr_day)


#df1.select(
 #   year("InsertedDate").alias('year'), 
  #  month("InsertedDate").alias('month'), 
   # dayofmonth("InsertedDate").alias('day')
#).show()

df1 = df1.withColumn("Customer Since", when(year("InsertedDate") == lit(curr_year), when(month("InsertedDate") == lit(curr_month), concat(lit(curr_day) - dayofmonth("InsertedDate"), lit(" days"))).otherwise(concat(lit(curr_month) - month("InsertedDate"), lit(" months")))).otherwise(concat(lit(curr_year) - year("InsertedDate"), lit(" years"))))
display(df1.limit(2))

# print( df1.select("InsertedDate").collect()[0][0])
# dfarray = np.array(df1.collect())
# print(dfarray[0][5])


In [6]:
df1.write.mode('overwrite').format('json').save('abfss://adlsynapsecontainer@adlstoragesynapse203.dfs.core.windows.net/synapse/workspaces/synapse-ws-203/Stage/temp/Customer Profile/')